Initialization pof the IPC Communication 
1. Open OPENLCA, activate the relevent database
2. Tools > Developper Tools > IPC Server > run the server with the correct port

In [1]:
import olca_ipc as ipc
import olca_schema as o
import pandas as pd
import numpy as np
import json
import uuid

from typing import Callable
###########################################################
client = ipc.Client(8080)

Technosphere matrice for 4 process with the 4 process 


In [14]:
technosphere = pd.DataFrame(
    data=[
        [1.0, -50.0, -1.0, 0.0],
        [-0.01, 1.0, -1.0, 0.0],
        [0.0, 0.0, 1.0, -1.0],
        [0.0, 0.0, 0.0, 100],
    ],
    columns=[
        "electricity production",
        "aluminium production",
        "aluminium foil production",
        "sandwitch package production",
    ],
    index=[
        "electricity [MJ]",
        "aluminium [kg]",
        "aluminium foil [kg]",
        "sandwitch package [Item(s)]",
    ],
)
print(technosphere)

                             electricity production  aluminium production  \
electricity [MJ]                               1.00                 -50.0   
aluminium [kg]                                -0.01                   1.0   
aluminium foil [kg]                            0.00                   0.0   
sandwitch package [Item(s)]                    0.00                   0.0   

                             aluminium foil production  \
electricity [MJ]                                  -1.0   
aluminium [kg]                                    -1.0   
aluminium foil [kg]                                1.0   
sandwitch package [Item(s)]                        0.0   

                             sandwitch package production  
electricity [MJ]                                      0.0  
aluminium [kg]                                        0.0  
aluminium foil [kg]                                  -1.0  
sandwitch package [Item(s)]                         100.0  


technosphere to biosphere 

In [15]:
interventions = pd.DataFrame(
    data=[
        [0.0, -5.0, 0.0, 0.0],
        [-0.5, 0.0, 0.0, 0.0],
        [3.0, 0.0, 0.0, 0.0],
        [2.0, 10.0, 0.0, 1.0],
    ],
    columns=technosphere.columns,
    index=[
        "bauxite [kg]",
        "crude oil [kg]",
        "CO2 [kg]",
        "solid waste [kg]",
    ],
)
print(interventions)

                  electricity production  aluminium production  \
bauxite [kg]                         0.0                  -5.0   
crude oil [kg]                      -0.5                   0.0   
CO2 [kg]                             3.0                   0.0   
solid waste [kg]                     2.0                  10.0   

                  aluminium foil production  sandwitch package production  
bauxite [kg]                            0.0                           0.0  
crude oil [kg]                          0.0                           0.0  
CO2 [kg]                                0.0                           0.0  
solid waste [kg]                        0.0                           1.0  


In [16]:
f = [
    0.0,
    0.0,
    0.0,
    10,
]
s = np.linalg.solve(technosphere.to_numpy(), f)
g = interventions.to_numpy() @ s
print(pd.DataFrame(g, index=interventions.index))

                      0
bauxite [kg]      -1.01
crude oil [kg]    -5.10
CO2 [kg]          30.60
solid waste [kg]  22.52


In [2]:
mass_units = o.new_unit_group("Mass units", "kg")
energy_units = o.new_unit_group("Energy units", "MJ")
counting_units = o.new_unit_group("Counting units", "Item(s)")
mass = o.new_flow_property("Mass", mass_units)
energy = o.new_flow_property("Energy", energy_units)
count = o.new_flow_property("Number of items", counting_units)

client.put_all(
    mass_units,
    energy_units,
    counting_units,
    mass,
    energy,
    count,
)
print(client.get(o.FlowProperty, name="Mass").to_json())

{
  "@type": "FlowProperty",
  "@id": "66c1e27d-b328-4404-b4d8-0093717446fb",
  "lastChange": "2023-11-22T14:26:32.211Z",
  "name": "Mass",
  "unitGroup": {
    "@type": "UnitGroup",
    "@id": "26637244-01e9-4ffb-94ba-949d9d9ae6b5",
    "name": "Mass units"
  },
  "version": "01.00.000"
}


In [18]:
def create_flow(
    row_label: str, fn: Callable[[str, o.FlowProperty], o.Flow]
) -> o.Flow:
    parts = row_label.split("[")
    name = parts[0].strip()
    unit = parts[1][0:-1].strip()
    match unit:
        case "kg":
            prop = mass
        case "MJ":
            prop = energy
        case "Item(s)":
            prop = count
    flow = fn(name, prop)
    client.put(flow)
    return flow


tech_flows = [create_flow(label, o.new_product) for label in technosphere.index]
envi_flows = [
    create_flow(label, o.new_elementary_flow) for label in interventions.index
]

In [23]:
def create_process(index: int, name: str) -> o.Process:
    process = o.new_process(name)

    def exchange(flow: o.Flow, value: float) -> o.Exchange | None:
        if value == 0:
            return None
        if value < 0:
            return o.new_input(process, flow, abs(value))
        else:
            return o.new_output(process, flow, value)

    for (i, tech_flow) in enumerate(tech_flows):
        value = technosphere.iat[i, index]
        e = exchange(tech_flow, value)
        if e and i == index:
            e.is_quantitative_reference = True

    for (i, envi_flow) in enumerate(envi_flows):
        value = interventions.iat[i, index]
        exchange(envi_flow, value)

    client.put(process)
    return process


processes = [
    create_process(index, name)
    for (index, name) in enumerate(technosphere.columns)
]

Calculation setup 1

In [20]:
setup = o.CalculationSetup(
    target=o.Ref(ref_type=o.RefType.Process, id=processes[3].id),
    unit=count.unit_group.ref_unit,  # "Item(s)"
    amount=10,
)
result = client.calculate(setup)
result.wait_until_ready()

inventory = result.get_total_flows()
print(
    pd.DataFrame(
        data=[
            (
                i.envi_flow.flow.name,
                i.envi_flow.is_input,
                i.amount,
                i.envi_flow.flow.ref_unit,
            )
            for i in inventory
        ],
        columns=["Flow", "Is input?", "Amount", "Unit"],
    )
)

result.dispose()

          Flow  Is input?  Amount Unit
0          CO2      False   30.60   kg
1  solid waste      False   22.52   kg
2    crude oil       True    5.10   kg
3      bauxite       True    1.01   kg


In [22]:
models = client.get_descriptors(o.ProductSystem)
for model in models:
    print(f"{model.name} :: {model.id}")


methods = client.get_descriptors(o.ImpactMethod)
for method in methods:
    print(f"{method.name} :: {method.id}")
method = methods[0]

Testing :: 5b745732-bb1b-4425-adca-60088e9d9201


Calculation setup 2


In [ ]:
setup = o.CalculationSetup(
    target=model,
    impact_method=method,
)
print(json.dumps(setup.to_dict(), indent=2))
##### Calculation ####
result = client.calculate(setup)
result.wait_until_ready()
impacts = result.get_total_impacts()
for i in impacts:
    assert i.impact_category
    print(f"{i.impact_category.name} {i.amount} {i.impact_category.ref_unit}")
result.dispose()



Parameters

In [ ]:
parameters = client.get_parameters(o.ProductSystem, model.id)
for param in parameters:
    print(f"parameter: {param.name} = {param.value}")
param = parameters[0]

Adjust the NRJ Intensity


In [ ]:

### TESTING ####
A = o.ParameterRedef(name='NRJ_Intensivity', value = 30)
print (A)
for param in parameters:
    print(f"parameter: {param.name} = {param.value}")
param = parameters[0]

In [ ]:
#### TESTING ###

A = param.context
print (A)

In [ ]:
for x in range(0, 10, 1):
    setup = o.CalculationSetup(
        target=model,
        impact_method=method,
        parameters=[
            o.ParameterRedef(name='NRJ_Intensivity', value=x,context=param.context)
        ],
    )
    result = client.calculate(setup)
    assert result
    result.wait_until_ready()
    impacts = result.get_total_impacts()
    for i in impacts:
        if i.impact_category.name == "Climate change, long term":
            print(f"{param.name}: {x} => {i.amount : .3f} kg CO2 eq")
    result.dispose()



In [ ]:
refs = client.get_descriptors(o.FlowProperty)
for ref in refs:
    print(ref.name)
# Area
# Area*time
# Duration
# Energy